<a href="https://colab.research.google.com/github/shandarmodabbir/Multi-Agent-Research-Assistant/blob/main/Multi_Agent_Research_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install google-search-results
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00


In [14]:
!pip install -q litellm python-dotenv
!pip install -q litellm langchain langchain-community python-dotenv

import os
from getpass import getpass
from litellm import completion

os.environ["GROQ_API_KEY"] = getpass(" Enter your GROQ_API_KEY: ")
api_key = os.environ["GROQ_API_KEY"]
api_base = "https://api.groq.com/openai/v1"

os.environ["SERPAPI_API_KEY"] = getpass(" Enter your SERPAPI_API_KEY: ")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
 Enter your GROQ_API_KEY: ··········
 Enter your SERPAPI_API_KEY: ··········


In [15]:
def query_groq(prompt, model="llama3-8b-8192"):
    response = completion(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        api_key=api_key,
        api_base=api_base,
        temperature=0.7,
        max_tokens=512
    )
    return response["choices"][0]["message"]["content"]

In [16]:
from langchain_community.utilities import SerpAPIWrapper

def get_research_material(query: str):
    search = SerpAPIWrapper(serpapi_api_key=os.environ["SERPAPI_API_KEY"])
    return search.run(query + " site:arxiv.org OR site:researchgate.net")


In [24]:
from langchain.prompts import PromptTemplate

def summarize_content(content: str):
    prompt = PromptTemplate.from_template("""
    Summarize the following content clearly and concisely for a research audience:

    {content}
    """)

    final_prompt = prompt.format(content=content)
    return query_groq(final_prompt)


In [18]:
def compare_sources(summaries: list):
    joined = "\n\n".join(summaries)
    prompt = f"""
    Compare and critique the following research summaries. Highlight differing views and any gaps:\n\n{joined}
    """
    return query_groq(prompt)


In [19]:
def generate_report(summaries, critique):
    joined = "\n\n".join(summaries)
    prompt = f"""
    Based on the summaries and critique below, write a clear research brief:

    === Summaries ===
    {joined}

    === Critique ===
    {critique}
    """
    return query_groq(prompt)

In [25]:
def run_pipeline(query: str):
    print("🔍 Searching for material...")
    results = get_research_material(query)

    print("🧾 Summarizing content...")

    summaries = [summarize_content(results)]

    print("⚖️ Comparing sources...")
    critique = compare_sources(summaries)

    print("🧑‍🏫 Generating report...")
    report = generate_report(summaries, critique)

    return report


In [26]:
prompt = "Summarize the latest advancements in reinforcement learning for robotics."
response = run_pipeline(prompt)
print("Response:\n", response)


🔍 Searching for material...
🧾 Summarizing content...
⚖️ Comparing sources...
🧑‍🏫 Generating report...
Response:
 Here is a clear research brief based on the summaries and critique:

**Title:** Deep Reinforcement Learning in Robotics: A Review of Recent Advances and Future Directions

**Objective:**

The objective of this research is to conduct a comprehensive review of recent studies on Deep Reinforcement Learning (DRL) and Inverse Reinforcement Learning in robotics, focusing on the past five years of research. The review aims to provide a balanced assessment of the strengths and limitations of DRL in robotics, highlighting its potential benefits, challenges, and competing approaches.

**Scope:**

The review will cover various application domains, including optimization in automation, and evaluate the real-world successes achieved with DRL in robotics. The research will also provide a comprehensive overview of RL techniques, including the development of novel artificial agents and the 

In [28]:
import gradio as gr

def gradio_interface(query):
    return run_pipeline(query)

gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Enter Research Topic", placeholder="e.g., Advances in Graph Neural Networks"),
    outputs=gr.Textbox(label="Generated Research Report"),
    title="🧠 Research Assistant",
    description="Enter a topic to generate a summarized research report using Groq + LangChain + SerpAPI."
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98df9459f36868165a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
